In [1]:
import numpy as np
import pandas as pd
import statsmodels as sm
import matplotlib.pyplot as plt
import scipy as sc


function for vec operator

In [13]:
def vec(mA):
    return np.asmatrix(mA.ravel('F')) # ravel flattens the inpute array into 1-dim
                                      # F specifies flatten by columns
                                      # asmatrix convert the input into a numpy matrix

Function for time-series or cross-sectional de-meaning

In [15]:
def Demean(mA, iAxis):
    
        if iAxis == 0:
            # when iAxis=0 substract mean alnong column
            mA = mA - np.mean(mA,axis=iAxis).reshape(1,int(np.shape(mA)[1-iAxis]))
        else:
            # when iAxis=0 substract mean alnong row
            mA = mA - np.mean(mA,axis=iAxis).reshape(int(np.shape(mA)[1-iAxis]),1)
        
        return mA

Test

Below is the function that can be used to generate the data

In [6]:
def GenrData (iSizeT, iSizeN, dAlpha, iS):  # set iS =1 when calling Number of "lagged" periods to skip or warm up before considering valid data.
    mErrors  = np.random.randn(iSizeN, iSizeT+iS+1)
    mDataY = np.zeros((iSizeN,iSizeT+iS+1))
    for t in range(1,iSizeT+iS+1):    
        mDataY[:,[t]] = dAlpha*mDataY[:,[t-1]] +  mErrors[:,[t]]
    return (np.transpose(mDataY[:,iS+1:]), np.transpose(mDataY[:,iS:iSizeT+iS]))# dim (iSizeT, iSizeN)

In [31]:
def FE(y,x): # function for the FE estimator
    # first demean the variables
    # since in the dgp they did not demean
    y_demean=np.asmatrix(Demean(y, 1))
    x_demean=np.asmatrix(Demean(x, 1))

    # FE estimator
    alpha_hat=np.linalg.inv(x_demean.T @ x_demean) @ x_demean.T @ y_demean

    return alpha_hat



This function provides FE-HPJ estimator, assuming that function that gives you FE estimator as inpute takes Y, X data only

In [18]:
def HPJ (mDataY, mDataX, dEstimator, Estimation): # input is dEstimator(FE_estimator) and  Estimation(FE function)
    
    iSizeT,iSizeN = np.shape(mDataY)
    iHalf = int(np.floor(iSizeT/2))
    

    dEstimator1 = Estimation (mDataY[:iHalf,:], mDataX[:iHalf,:])
    
    dEstimator2 = Estimation (mDataY[iHalf:,:], mDataX[:iHalf,:])
    
    dHPJ = 2*dEstimator-0.5*(dEstimator1+dEstimator2)
    
    
    return (dHPJ,dEstimator1,dEstimator2) 

The following skeleton of bootstrap can be used to generate RDWB CI for both FE and HPJ (ising any type residuals). Output is a $[P\times 2]$ vector of indicator variables whether corresponding $H_{0}$ is rejected or not. $P$ can be a general number if you want to consider testing many $H_{0}$ at the same time, i.e. construct confidence intervals (parts 2 and 3 in Section 3).

In [ ]:
# def BootstrapMC (mResiduals, vY0, iB, vNull, dLevel,vEst,dAlpha_O_bootstrap):
    
#     iSizeT, iSizeN = np.shape(mResiduals)
#     iSizeP = np.shape(vNull)[0]
#     iSizeEst = int(np.shape(vEst)[0])
    
#     iUnitLow = int(np.floor((dLevel/2)*iB))
#     iUnitHigh = int(np.floor((1-dLevel/2)*iB))
    
#     mResB = np.empty((iB,iSizeEst))
#     mResB[0,:] = vEst.T
    
#     for b in range(1,iB):
        
#         #this should give Rademacher weights. Double check.
#         mWeights = np.random.choice([-1, 1], size=mResiduals.shape)
        
#         mErrors_B = np.multiply((mResiduals,mWeights))
        
#         # modife GenrData function to take as inputs initial condition,and matrix of residuals for error terms
#         mDataY_B, mDataY_lag_B = GenrDataBootstrap (iSizeT, iSizeN, dAlpha_O_bootstrap,iS,vY0,mErrors_B)
        
#         dFE_B = FE (mDataY_B, mDataY_lag_B)
#         dFE_HPJ_B = HPJ (mDataY_B, mDataY_lag_B,dFE_B,FE)
        
#         mResB[b,0] = dFE_B
#         mResB[b,1] = dFE_HPJ_B 
    
#     mReject = np.zeros((iSizeEst,iSizeP))
#     mResB_Sorted = np.sort(mResB, axis=0)
#     for k in range(0,iSizeEst):
               
#         dLowQ = mResB_Sorted[iUnitLow,k]
#         dHighQ = mResB_Sorted[iUnitHigh,k] 
#         dLow = dLowQ
#         dHigh = dHighQ
    
#         mReject[k,:] = np.where((vNull < dHigh) & (vNull> dLow),0,1)
                   
#     return mReject.T

Below I summarize the skeleton for the function that performs Monte Carlo study

In [ ]:
def MC_study (iSizeT, dKappa,dGamma, iM,iB):
    
    
    # here you store your estimation results
    mResultsEstimation = np.zeros((iM,2))
    # here you store your testing results for any choice of P. Standard P=1
    iP = 1
    mResultsTesting = np.zeros((iM,2*iP)) 
    
    
    for m in range (0,iM):
        
        # Remember that the true value is a function of T,\gamma,\kappa
        dAlpha_0 =  np.exp(-iSizeT**-dGamma)
        
        iSizeN = int(iSizeT**(1/dKappa))
        
        mDataY, mDataY_lag = GenrData (iSizeT, iSizeN, dAlpha_0,iS)
        
        dFE = FE (mDataY, mDataY_lag)
        dFE_HPJ = HPJ (mDataY, mDataY_lag,dFE,FE)
        
    
        mResiduals_FE = mDataY-mDataY_lag@dFE
        
        #make sure that your residuals sum up to zero! This is not automatically satisfied for the HPJ method
        mResiduals_HPJ = mDataY-mDataY_lag@dFE_HPJ
        # to ensure the residua sum up to 0
        mResiduals_HPJ = mResiduals_HPJ - np.mean(mResiduals_HPJ, axis=0, keepdims=True)

        # vEst = np.vstack((dFE,dFE_HPJ))
        
        # #this can be either FE or HPJ, or true value of alpha_{0} that you used to generate the true DGPJ
        # dAlpha_O_bootstrap = 
        
        # #this can be either residuals from FE or HPJ estimation
        # mResiduals_B = 
        # vNull = dAlpha_0
        # mReject = BootstrapMC (mResiduals_B, vY0, iB, vNull, dLevel,vEst,dAlpha_O_bootstrap)
        

        # it is convenient to store estimates in deviations from the true values
        mResultsEstimation[m,0] = dFE - dAlpha_0
        mResultsEstimation[m,1] = dHPJ - dAlpha_0
        
    #bias 
    vBias = np.mean(mResultsEstimation[m,0], axis=0)
    
    #RMSE 
    vRMSE = np.sqrt(np.mean(np.power(mResultsEstimation,2),axis=0))

    return vBias,vRMSE
    
    

    

In [ ]:
import numpy as np

# Generate data for AR(1) process
def GenrData(iSizeT, iSizeN, dAlpha, iS):
    mErrors = np.random.randn(iSizeN, iSizeT + iS + 1)
    mDataY = np.zeros((iSizeN, iSizeT + iS + 1))
    for t in range(1, iSizeT + iS + 1):
        mDataY[:, [t]] = dAlpha * mDataY[:, [t - 1]] + mErrors[:, [t]]
    return mDataY[:, iS + 1:], mDataY[:, iS:iSizeT + iS]

# Recursive Design Wild Bootstrap
def RWB(mResiduals, dAlpha_FE, vY0, iB):
    iSizeN, iSizeT = mResiduals.shape
    bootstrap_estimates = []

    for b in range(iB):
        # Generate Rademacher weights
        mWeights = np.random.choice([-1, 1], size=mResiduals.shape)
        mErrors_B = mWeights * mResiduals  # Bootstrap errors

        Simulate bootstrap DGP
        mDataY_B = np.zeros((iSizeN, iSizeT + 1))
        mDataY_B[:, 0] = vY0
        for t in range(1, iSizeT + 1):
            mDataY_B[:, t] = dAlpha_FE * mDataY_B[:, t - 1] + mErrors_B[:, t - 1]
        
        # Estimate FE from bootstrap sample
        dAlpha_B = np.sum(mDataY_B[:, 1:] * mDataY_B[:, :-1]) / np.sum(mDataY_B[:, :-1]**2)
        bootstrap_estimates.append(dAlpha_B)

    return np.array(bootstrap_estimates)

# Monte Carlo Study
def MC_study(iSizeT, dKappa, dGamma, iM, iB, dLevel):
    mResultsEstimation = np.zeros((iM, 2))  # FE and HPJ
    mRejectionFreq = np.zeros(iM)  # Rejection frequencies
    iSizeN = int(iSizeT**(1/dKappa))
    dAlpha_0 = np.exp(-iSizeT**-dGamma)

    for m in range(iM):
        # Generate true data
        mDataY, mDataY_lag = GenrData(iSizeT, iSizeN, dAlpha_0, iS=1)
        dAlpha_FE = np.sum(mDataY * mDataY_lag) / np.sum(mDataY_lag**2)

        # FE residuals
        mResiduals_FE = mDataY - dAlpha_FE * mDataY_lag

        # Bootstrap
        bootstrap_estimates = RWB(mResiduals_FE, dAlpha_FE, mDataY[:, 0], iB)
        
        # Confidence intervals
        CI_low = np.percentile(bootstrap_estimates, dLevel / 2 * 100)
        CI_high = np.percentile(bootstrap_estimates, (1 - dLevel / 2) * 100)

        # Store results
        mResultsEstimation[m, 0] = dAlpha_FE - dAlpha_0
        mRejectionFreq[m] = int(dAlpha_0 < CI_low or dAlpha_0 > CI_high)

    # Bias and RMSE
    vBias = np.mean(mResultsEstimation, axis=0)
    vRMSE = np.sqrt(np.mean(mResultsEstimation**2, axis=0))
    rejection_rate = np.mean(mRejectionFreq)

    return vBias, vRMSE, rejection_rate

# Parameters
T_values = [20, 50, 100]
kappa_values = [0.6, 0.8, 1.0]
gamma_values = [0.75, 1.0, 1.25]
iM = 40
iB = 199
dLevel = 0.05

# Run the study for all combinations of (T, kappa, gamma)
results = []
for T in T_values:
    for kappa in kappa_values:
        for gamma in gamma_values:
            vBias, vRMSE, rejection_rate = MC_study(T, kappa, gamma, iM, iB, dLevel)
            results.append((T, kappa, gamma, vBias, vRMSE, rejection_rate))



Test


In [246]:
def vec(mA):
    return np.asmatrix(mA.ravel('F')).T # ravel flattens the inpute array into 1-dim
                                      # F specifies flatten by columns
                                      # asmatrix convert the input into a numpy matrix

def Demean(mA, iAxis):
    
        if iAxis == 0:
            # when iAxis=0 substract mean alnong column
            mA = mA - np.mean(mA,axis=iAxis).reshape(1,int(np.shape(mA)[1-iAxis]))
        else:
            # when iAxis=0 substract mean alnong row
            mA = mA - np.mean(mA,axis=iAxis).reshape(int(np.shape(mA)[1-iAxis]),1)
        
        return mA
def GenrData (iSizeT, iSizeN, dAlpha, iS):  # set iS =1 when calling Number of "lagged" periods to skip or warm up before considering valid data.
    mErrors  = np.random.randn(iSizeN, iSizeT+iS+1)
    mDataY = np.zeros((iSizeN,iSizeT+iS+1))
    for t in range(1,iSizeT+iS+1):    
        mDataY[:,[t]] = dAlpha*mDataY[:,[t-1]] +  mErrors[:,[t]]
    return (np.transpose(mDataY[:,iS+1:]), np.transpose(mDataY[:,iS:iSizeT+iS]))# dim (iSizeT, iSizeN)

def FE(y,x): # function for the FE estimator
    # first demean the variables
    # since in the dgp they did not demean
    y_demeaned=vec(Demean(y, 0))
    x_demeaned=vec(Demean(x, 0))

    # FE estimator
    alpha_hat=np.linalg.inv(x_demeaned.T @ x_demeaned) @ x_demeaned.T @ y_demeaned

    return alpha_hat 

def HPJ (mDataY, mDataX, dEstimator, Estimation): # input is dEstimator(FE_estimator) and  Estimation(FE function)
    
    iSizeT,iSizeN = np.shape(mDataY)
    iHalf = int(np.floor(iSizeT/2))
    

    dEstimator1 = Estimation (mDataY[:iHalf,:], mDataX[:iHalf,:])
    
    dEstimator2 = Estimation (mDataY[iHalf:,:], mDataX[iHalf:,:])
    
    dHPJ = 2*dEstimator-0.5*(dEstimator1+dEstimator2)
    
    
    return (dHPJ,dEstimator1,dEstimator2) 

def MC_study (iSizeT, dKappa,dGamma, iM,iB):
    
    np.random.seed(42)
    # here you store your estimation results
    mResultsEstimation = np.asmatrix(np.zeros((iM,2)))
    # here you store your testing results for any choice of P. Standard P=1
    iP = 1
    mResultsTesting = np.zeros((iM,2*iP)) 
    
    
    for m in range (0,iM):
        
        # Remember that the true value is a function of T,\gamma,\kappa
        dAlpha_0 =  np.exp(-iSizeT**-dGamma)
        
        iSizeN = int(iSizeT**(1/dKappa))
        
        mDataY, mDataY_lag = GenrData (iSizeT, iSizeN, dAlpha_0,iS)
        
        dFE = FE (mDataY, mDataY_lag)
        dFE_HPJ = HPJ (mDataY, mDataY_lag,dFE,FE)
        dHPJ=dFE_HPJ[0]
        
    
        # mResiduals_FE = mDataY-mDataY_lag@dFE
        
        #make sure that your residuals sum up to zero! This is not automatically satisfied for the HPJ method
        # mResiduals_HPJ = mDataY-mDataY_lag@dFE_HPJ
        # to ensure the residua sum up to 0
         #mResiduals_HPJ = mResiduals_HPJ - np.mean(mResiduals_HPJ, axis=0, keepdims=True)

        # vEst = np.vstack((dFE,dFE_HPJ))
        
        # #this can be either FE or HPJ, or true value of alpha_{0} that you used to generate the true DGPJ
        # dAlpha_O_bootstrap = 
        
        # #this can be either residuals from FE or HPJ estimation
        # mResiduals_B = 
        # vNull = dAlpha_0
        # mReject = BootstrapMC (mResiduals_B, vY0, iB, vNull, dLevel,vEst,dAlpha_O_bootstrap)
        

        # it is convenient to store estimates in deviations from the true values
        mResultsEstimation[m,0] = dFE - dAlpha_0
        mResultsEstimation[m,1] = dHPJ - dAlpha_0
        
    #bias 
    # vBias =np.mean(mResultsEstimation[:, 0])
    # Bias for FE (first column of mResultsEstimation)
    vBias_FE = np.mean(mResultsEstimation[:, 0])

    # Bias for HPJ (second column of mResultsEstimation)
    vBias_HPJ = np.mean(mResultsEstimation[:, 1])
    #RMSE 
    vRMSE = np.sqrt(np.mean(np.power(mResultsEstimation,2),axis=0))

    return iSizeN,vBias_FE,vBias_HPJ, vRMSE

    # Parameters
T_values = [20, 50, 100]
kappa_values = [0.6, 0.8, 1.0]
gamma_values = [0.75, 1.0, 1.25]
iM = 40
iB = 199
iS=1
# Run the study for all combinations of (T, kappa, gamma)
results = []
for T in T_values:
    for kappa in kappa_values:
        for gamma in gamma_values:
            iSizeN,vBias_FE, vBias_HPJ,vRMSE = MC_study(T, kappa, gamma, iM, iB)
            results.append((iSizeN,T, kappa, gamma, vRMSE,vBias_FE, vBias_HPJ))
            print(f"When T={T},kappa={kappa},gsmma={gamma},RMSE for FE estimator={vRMSE[0,0]:.4f},RMSE for HPJ estimator={vRMSE[0,1]:.4f}, MC Bias for FE estimator={vBias_FE:.4f}, MC Bias for HPJ estimator={vBias_HPJ:.4f}")



/var/folders/37/1816qp916w797616r1v9n33m0000gn/T/ipykernel_14280/3768229169.py:92: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  mResultsEstimation[m,0] = dFE - dAlpha_0
/var/folders/37/1816qp916w797616r1v9n33m0000gn/T/ipykernel_14280/3768229169.py:93: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  mResultsEstimation[m,1] = dHPJ - dAlpha_0


When T=20,kappa=0.6,gsmma=0.75,RMSE for FE estimator=0.1373,RMSE for HPJ estimator=0.0208, MC Bias for FE estimator=-0.1367, MC Bias for HPJ estimator=-0.0083
When T=20,kappa=0.6,gsmma=1.0,RMSE for FE estimator=0.1479,RMSE for HPJ estimator=0.0247, MC Bias for FE estimator=-0.1475, MC Bias for HPJ estimator=-0.0166
When T=20,kappa=0.6,gsmma=1.25,RMSE for FE estimator=0.1483,RMSE for HPJ estimator=0.0229, MC Bias for FE estimator=-0.1479, MC Bias for HPJ estimator=-0.0147
When T=20,kappa=0.8,gsmma=0.75,RMSE for FE estimator=0.1440,RMSE for HPJ estimator=0.0416, MC Bias for FE estimator=-0.1417, MC Bias for HPJ estimator=-0.0168
When T=20,kappa=0.8,gsmma=1.0,RMSE for FE estimator=0.1553,RMSE for HPJ estimator=0.0489, MC Bias for FE estimator=-0.1535, MC Bias for HPJ estimator=-0.0276
When T=20,kappa=0.8,gsmma=1.25,RMSE for FE estimator=0.1561,RMSE for HPJ estimator=0.0488, MC Bias for FE estimator=-0.1544, MC Bias for HPJ estimator=-0.0264
When T=20,kappa=1.0,gsmma=0.75,RMSE for FE estim

In [247]:
columns = [
    "N",
    "T",
    r" κ  ",
    r"γ ",
    "RMSE for FE estimator",
    "RMSE for HPJ estimator",
    "MC Bias for FE estimator",
    "MC Bias for HPJ estimator",
]
data = [
    (iSizeN,T, kappa, gamma, rmse[0, 0], rmse[0, 1], bias_fe, bias_hpj)
    for iSizeN,T, kappa, gamma, rmse, bias_fe, bias_hpj in results
]
df = pd.DataFrame(data, columns=columns)
df=df.round(4).reset_index(drop=True)


In [248]:
df

,N,T,κ,γ,RMSE for FE estimator,RMSE for HPJ estimator,MC Bias for FE estimator,MC Bias for HPJ estimator
0,147,20,0.6,0.75,0.1373,0.0208,-0.1367,-0.0083
1,147,20,0.6,1.00,0.1479,0.0247,-0.1475,-0.0166
2,147,20,0.6,1.25,0.1483,0.0229,-0.1479,-0.0147
3,42,20,0.8,0.75,0.1440,0.0416,-0.1417,-0.0168
4,42,20,0.8,1.00,0.1553,0.0489,-0.1535,-0.0276
5,42,20,0.8,1.25,0.1561,0.0488,-0.1544,-0.0264
6,20,20,1.0,0.75,0.1530,0.0674,-0.1476,-0.0215
7,20,20,1.0,1.00,0.1628,0.0701,-0.1582,-0.0306
8,20,20,1.0,1.25,0.1627,0.0671,-0.1586,-0.0288
9,678,50,0.6,0.75,0.0562,0.0051,-0.0561,-0.0008


In [237]:
results

[(20,
  0.6,
  0.75,
  matrix([[0.13731882, 0.02078563]]),
  -0.13672150767578595,
  -0.008309052510021083),
 (20,
  0.6,
  1.0,
  matrix([[0.14793941, 0.02470988]]),
  -0.1474634589844333,
  -0.016613590089757747),
 (20,
  0.6,
  1.25,
  matrix([[0.14829456, 0.02293965]]),
  -0.14785520753473375,
  -0.014744096754926748),
 (20,
  0.8,
  0.75,
  matrix([[0.1440309 , 0.04155484]]),
  -0.14172150587951468,
  -0.016786645937454246),
 (20,
  0.8,
  1.0,
  matrix([[0.15533845, 0.04894503]]),
  -0.15347254461259613,
  -0.027621028347139937),
 (20,
  0.8,
  1.25,
  matrix([[0.15609695, 0.04876239]]),
  -0.15437780833458795,
  -0.026369571840918715),
 (20,
  1.0,
  0.75,
  matrix([[0.15304279, 0.06736083]]),
  -0.14758466932430822,
  -0.02151861833312846),
 (20,
  1.0,
  1.0,
  matrix([[0.16280713, 0.07009647]]),
  -0.15819062070215312,
  -0.03063706949601795),
 (20,
  1.0,
  1.25,
  matrix([[0.16271248, 0.06710271]]),
  -0.15857292195872935,
  -0.028803643698335955),
 (50,
  0.6,
  0.75,
  ma